In [4]:
import pydicom as dicom
import numpy as np
import sys
import cv2 as cv
import imutils
import json
from math import sqrt, pow

In [5]:
def linearTransform(x, minimum, maximum, a, b):
    return ((b - a) * ((x - minimum) / (maximum - minimum)) + a)


def getSegmentedPixelColor(value, minimum, maximum):
    if value >= minimum and value <= maximum:
        return linearTransform(value, 0, 255, minimum, maximum)
    return 0


def getSegmentedBGR(pixelArray, rows, cols, minimum, maximum):
    image = np.zeros((rows, cols, 3), np.uint8)
    for i in range(rows):
        for j in range(cols):
            image[i][j][0] = image[i][j][1] = image[i][j][2] = getSegmentedPixelColor(
                pixelArray[i][j], minimum, maximum)
    return image

In [6]:
def executeSegmentation(src, minimum, maximum, output):
    ds = dicom.dcmread(src)

    rows = ds.Rows
    cols = ds.Columns

    patientId = ds.PatientID
    im = getSegmentedBGR(ds.pixel_array, rows, cols, minimum, maximum)

    im = cv.cvtColor(im, cv.COLOR_BGR2GRAY)

    cv.imwrite(output, im)

In [3]:
def executeFilters(src, out, operation, kernelParam=2):
    img = cv.imread(src, 0)

    if operation is 'erode':
        kernelSize = int(kernelParam)
        kernel = np.ones((kernelSize,kernelSize),np.uint8)
        img = cv.erode(img, kernel, iterations=1)

    if operation is 'dilate':
        kernelSize = int(kernelParam)
        kernel = np.ones((kernelSize,kernelSize),np.uint8)
        img = cv.dilate(img,kernel,iterations = 1)

    cv.imwrite(out, img)

In [139]:
def executeGrabFeatures(src, out, minArea=5, drawLabel=True):
    img = cv.imread(src, 0)
    labeled = cv.cvtColor(img, 0, cv.COLOR_GRAY2BGR)
    contours, hierarchy = cv.findContours(img, cv.RETR_EXTERNAL,  cv.CHAIN_APPROX_NONE)
    # Sort by larger areas
    contoursFeatures = []
    cnts = sorted(contours, key=cv.contourArea, reverse=True)
    cntsToDraw = []
    for i in range(0, len(cnts)):
        if cv.contourArea(cnts[i]) >= minArea:
            cntsToDraw.append(cnts[i])
            features = {}
            M = cv.moments(cnts[i])
            centroid = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
            features['xCentroid'] = centroid[0]
            features['yCentroid'] = centroid[1]
            features['arcLength'] = cv.arcLength(cnts[i], True)
            features['area'] = cv.contourArea(cnts[i])
            if drawLabel:
                cv.putText(labeled, str(i), (centroid[0], centroid[1]), cv.FONT_HERSHEY_SIMPLEX, 1, (209, 80, 0, 255), 2)
            try:
                # Calculate eccentricity
                (x, y), (MA, ma), angle = cv.fitEllipse(cnts[i])
                a = ma/2
                b = MA/2

                if (a > b):
                    eccentricity = sqrt(pow(a, 2)-pow(b, 2))
                    eccentricity = round(eccentricity/a, 2)
                else:
                    eccentricity = sqrt(pow(b, 2)-pow(a, 2))
                    eccentricity = round(eccentricity/b, 2)

                features['eccentricity'] = eccentricity
            except:
                features['eccentricity'] = -1
            contoursFeatures.append(features)
    f = open(out, 'w')
    f.write(json.dumps(contoursFeatures))
    if drawLabel:
        #cv.drawContours(labeled, cntsToDraw, -1, (0, 0,255), 3) 
        cv.imwrite(src.replace('.png', 'labeled.png'), labeled)

In [148]:
def execute(src):
    executeSegmentation(src, 42, 400, './seg.png')
    executeFilters('./seg.png', './erode.png', 'erode', 2)
    executeFilters('./erode.png', './dilate.png', 'dilate')
    executeGrabFeatures('./dilate.png', 'features.json')

In [149]:
execute('./ANTERIOR002_DS.dcm')